In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
from pathlib import Path
import os
import sys
sys.path.insert(0, str(Path(os.getcwd()).resolve().parent.parent))
from src.utils.tools import create_dummies

list_var_num: list = ['Duree_credit', 'Montant_credit', 'Age']
list_var_cat: list = ['Objet_credit', 'Historique_credit', 'Epargne', 'Anciennete_emploi', 
           'Situation_familiale', 'Anciennete_domicile', 'Nb_credits', 'Biens', 
           'Type_emploi', 'Comptes', 'Taux_effort', 'Autres_credits', 'Statut_domicile', 
           'Garanties', 'Telephone', 'Nb_pers_charge', 'Etranger']
target: str = "Cible"

path = Path("../../data/processed/credit_scoring_with_labels.parquet").resolve() # chemin vers fichier csv
df = pd.read_parquet(path)
df[list_var_cat] = df[list_var_cat].astype("category")

### 4. Discrétisation des variables continues  _(bonus)_

<div class="alert alert-block alert-warning">
<b> Question 23:</b> Pourquoi discrétiser ? 
</div>

<div class="alert alert-block alert-success">
<b> Réponse:</b> Cela peut permettre de capter les liens non linéaires qui peuvent exister entre la variable explicative continue et la variable cible
</div>

<div class="alert alert-block alert-warning">
<b> Question 24:</b> Pour quel(s) type(s) de modélisations la discrétisation est-t-elle le plus utile ? Le moins utile ?
</div>

<div class="alert alert-block alert-success">
  <b>Réponse :</b>
  <ul>
    <li>Le plus utile : La régression logistique, KNN</li>
    <li>Le moins utile : La discrétisation des variables continues n'est <b>pas nécessaire</b> pour les <b>modèles à base d'arbres</b> puisque par construction, ils feront eux-mêmes les découpages (i.e. la discrétisation). Néanmoins, cela peut rester intéressant</li>
  </ul>
</div>


<div class="alert alert-block alert-info">
  <b>Méthodes de discrétisation des variables continues :</b>
  <a href="https://eric.univ-lyon2.fr/ricco/cours/slides/discretisation.pdf">
    https://eric.univ-lyon2.fr/

### 5. Préparation à la modélisation

<div class="alert alert-block alert-info">
  <b>⚠️ Remarque très importante :</b>
  pensez à utiliser la même séparation apprentissage/test pour TOUS vos modèles afin de les rendre comparables entre eux.
</div>


#### 5.1 Séparation Apprentissage/Test

<div class="alert alert-block alert-warning">
<b> Question 26:</b> Construisez nos deux paires d'échantillons en respectant les proportions suivantes :
<br>- 80% pour l'échantillon d'apprentissage
<br>- 20% pour l'échantillon de test
</div>

<div class="alert alert-block alert-info">
<b>ℹ️ Aide :</b> utilisez la fonction <code>train_test_split</code> de la library <code>sklearn.model_selection</code>
</div>

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df, df[target],
                                                    test_size=0.2, stratify=df[target]) 
del X_train[target], X_test[target] # On retire la variable cible 

In [5]:
print(f"Echantillon train: Part y=1 : {y_train.astype(int).mean()}")
print(f"Echantillon test: Part y=1 : {y_test.astype(int).mean()}")

Echantillon train: Part y=1 : 0.3
Echantillon test: Part y=1 : 0.3


#### 5.2 Gestion des valeurs manquantes

<div class="alert alert-block alert-warning">
<b> Question 24:</b> Etudier le nombre de valeurs manquantes pour tous les champs. Puis faites un print des champs contenant des valeurs manquantes.
</div>

<div class="alert alert-block alert-success">
<b> Réponse:</b>
</div>

In [6]:
# Pour covariables
for name, dataframe in {"X_train": X_train, "X_test": X_test}.items():
    tmp = dataframe.isna().sum().reset_index().rename(columns={'index':'Field',0:'Nb_missing'})
    list_with_nan =  tmp.loc[tmp['Nb_missing']>0,'Field'].tolist()
    if len(list_with_nan)>0:
        print(f"{name} // Liste des variables contenant des valeurs manquantes {list_with_nan}")
# Pour variable cible
for name, values in {"y_train": y_train, "y_test": y_test}.items():
    nb_missing_values = values.isna().sum()
    if nb_missing_values>0:
        print(f"{name}: {nb_missing_values} valeurs manquantes")

<div class="alert alert-block alert-warning">
<b> Question 24:</b> Citez quelques méthodes pour traiter les valeurs manquantes (pour les variables explicatives)
</div>

<div class="alert alert-block alert-success">
  <p><b>Réponse :</b></p>
  <table>
    <thead>
      <tr>
        <th>Méthode</th>
        <th>Description</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Suppression</td>
        <td>Retirer les lignes contenant des valeurs manquantes ou supprimer la variable si trop de valeurs sont manquantes.</td>
      </tr>
      <tr>
        <td>Imputation par valeur statistique</td>
        <td>Remplacer les valeurs manquantes par une statistique (moyenne pour les variables numériques, médiane ou mode pour les variables catégorielles).</td>
      </tr>
      <tr>
        <td>Imputation par modèle prédictif</td>
        <td>Prédire la valeur manquante à l'aide d'un modèle (par exemple, régression ou k-Nearest Neighbors) basé sur les autres variables.</td>
      </tr>
      <tr>
        <td>Imputation multiple</td>
        <td>Utiliser des méthodes comme MICE (Multiple Imputation by Chained Equations) pour générer plusieurs jeux d’imputations et prendre en compte l’incertitude.</td>
      </tr>
      <tr>
        <td>Algorithmes d'apprentissage automatique</td>
        <td>Recourir à des modèles capables de gérer directement les valeurs manquantes (par exemple, certains arbres de décision ou forêts aléatoires).</td>
      </tr>
    </tbody>
  </table>
</div>


#### 5.2 Encoding des variables catégorielles

<div class="alert alert-block alert-warning">
<b> Question 25:</b> Citez au moins 2 raisons pour lesquelles il est important de réaliser un encoding des variables catégorielles
</div>

<div class="alert alert-block alert-success">
  <p><b>Réponse :</b></p>
  <table>
    <thead>
      <tr>
        <th>Raison</th>
        <th>Détails</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Compatibilité avec les algorithmes de ML</td>
        <td>Un grand nombre d'algorithmes n'autorisent que des variables numériques.</td>
      </tr>
      <tr>
        <td>Éviter une relation d'ordre inappropriée</td>
        <td>Par exemple, 1 pour « rouge », 2 pour « bleu », 3 pour « vert » risque de créer une hiérarchie ou une distance artificielle entre elles. L'encodage en dummies évite ce biais en créant une colonne binaire pour chaque catégorie.</td>
      </tr>
      <tr>
        <td>Faciliter/Rendre possible l'interprétation des résultats</td>
        <td></td>
      </tr>
    </tbody>
  </table>
</div>


#### 5.2.1 One hot encoding

<div class="alert alert-block alert-info">
<b>🔔 Rappel :</b> Le One hot encoding consiste à transformer une variable catégorielle en plusieurs colonnes binaires, chacune indiquant la présence (1) ou l'absence (0) d'une catégorie donnée pour chaque observation
</div>

<div class="alert alert-block alert-warning">
<b> Question 26:</b> Que faire si ma variable à dummiser a un grand nombre de modalités distinctes ?
</div>

<div class="alert alert-block alert-success">
  <b>Réponse :</b>
  <ul>
    <li>réduire le nombre de modalités</li>
    <li>adopter une autre méthode d'encoding</li>
  </ul>
</div>

<div class="alert alert-block alert-warning">
<b> Question 27:</b> Appliquer le One Hot Encoding aux variables catégorielles du dataset
</div>

<div class="alert alert-block alert-info">
<b>ℹ️ Aide :</b> utilisez la fonction <code>create_dummies</code>
</div>

In [7]:
# Rappel du nombre de modalités pour chaque variable catégorielle
print("Jeu de données train:")
print(X_train[list_var_cat].nunique())

Jeu de données train:
Objet_credit           10
Historique_credit       5
Epargne                 5
Anciennete_emploi       5
Situation_familiale     4
Anciennete_domicile     4
Nb_credits              4
Biens                   4
Type_emploi             4
Comptes                 4
Taux_effort             4
Autres_credits          3
Statut_domicile         3
Garanties               3
Telephone               2
Nb_pers_charge          2
Etranger                2
dtype: int64


In [8]:
print("Jeu de données test:")
print(X_test[list_var_cat].nunique())

Jeu de données test:
Objet_credit           9
Historique_credit      5
Epargne                5
Anciennete_emploi      5
Situation_familiale    4
Anciennete_domicile    4
Nb_credits             4
Biens                  4
Type_emploi            4
Comptes                4
Taux_effort            4
Autres_credits         3
Statut_domicile        3
Garanties              3
Telephone              2
Nb_pers_charge         2
Etranger               2
dtype: int64


In [9]:
X_train_with_dummies = create_dummies(dataframe= X_train,list_var_cat= list_var_cat, drop_original_var_cat= True) # apprentissage
X_test_with_dummies = create_dummies(dataframe= X_test,list_var_cat= list_var_cat, drop_original_var_cat= True) # test

In [10]:
# On vérifie que les mêmes DUMMY ont été créés pour les deux jeux de données (sinon il faudra traiter les différences)
dummies_train = [field for field in X_train_with_dummies.columns if field.startswith("DUMMY_")]
dummies_test = [field for field in X_train_with_dummies.columns if field.startswith("DUMMY_")]
not_in_train = list(set(dummies_test)-set(dummies_train))
not_in_test = list(set(dummies_test)-set(dummies_train))
if not_in_train:
    print("Champs présents dans X_test mais pas dans X_train: {not_in_train}")
if not_in_test:
    print("Champs présents dans X_test mais pas dans X_train: {not_in_test}")

In [11]:
# Export des données
path = Path(f"../../data/processed").resolve()
X_train_with_dummies.to_parquet(f"{path}/X_train.parquet")
X_test_with_dummies.to_parquet(f"{path}/X_test.parquet")
y_train.to_frame().to_parquet(f"{path}/y_train.parquet")
y_test.to_frame().to_parquet(f"{path}/y_test.parquet")

#### 5.2.2 Autres méthode d'encoding

<div class="alert alert-block alert-warning">
<b> Question 26:</b> Quelle(s) méthode(s) adopteriez-vous pour réduire le nombre de modalités ?
</div>

<div class="alert alert-block alert-success">
  <b>Réponse :</b></p>
  <table>
    <thead>
      <tr>
        <th>Approche</th>
        <th>Détail</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Gestion des modalités rares</td>
        <td>Regrouper les modalités rares.</td>
      </tr>
      <tr>
        <td>Approche experte</td>
        <td>Regrouper intelligemment certaines modalités.</td>
      </tr>
      <tr>
        <td>Approche stat</td>
        <td>Identifier et regrouper certaines modalités entre elles. Regrouper les modalités qui apportent la même information pour expliquer la cible.</td>
      </tr>
    </tbody>
  </table>
</div>


<div class="alert alert-block alert-warning">
<b> Question 27:</b> Quelles autres méthodes d'encoding des variables catégorielles (autre que one hot encoding) connaissez-vous ?
</div>

<div class="alert alert-block alert-success">
  <p><b>Réponse :</b></p>
  <table>
    <thead>
      <tr>
        <th>Méthode</th>
        <th>Détail</th>
      </tr>
    </thead>
    <tbody>
      <tr>
        <td>Target Encoding</td>
        <td>Remplace chaque catégorie par une statistique (souvent la moyenne) de la variable cible pour cette catégorie. Cela permet d'introduire des informations sur la relation entre la catégorie et la cible, mais nécessite des précautions (ex. : validation croisée) pour éviter le surapprentissage.</td>
      </tr>
      <tr>
        <td>Frequency Encoding</td>
        <td>Remplace chaque catégorie par sa fréquence (ou le pourcentage) d'apparition dans le dataset.</td>
      </tr>
      <tr>
        <td>Leave-One-Out Encoding</td>
        <td>Variante du target encoding, cette méthode calcule la statistique pour une observation donnée en excluant cette observation. Elle permet de réduire le risque de surapprentissage tout en tirant profit de l'information de la cible.</td>
      </tr>
    </tbody>
  </table>
</div>
